# Data Cleaning steps provided below
Original data birdnames.csv was generated by copying all of wikipedia page listed below and filtering out just the names. This could probably be scripted as well but didnt feel like it

https://en.wikipedia.org/wiki/List_of_birds_of_Georgia_(U.S._state)

In [16]:
import pandas as pd
data = pd.read_csv("birdnames.csv", header = None)
data.drop(1, inplace = True, axis = 1)
data.head(5)

,0
0,"Black-bellied whistling-duck, Dendrocygna autu..."
1,"Fulvous whistling-duck, Dendrocygna bicolor (R)"
2,"Snow goose, Anser caerulescens"
3,"Ross's goose, Anser rossii"
4,"Greater white-fronted goose, Anser albifrons"


In [27]:
names = []
genus = []
info = []
for i in range(len(data[0])):
    value = data[0][i].split(",")
    names.append(value[0])
    splitter = value[1].split("\xa0")
    if len(splitter) != 3:
        splitter.append("None")
    if splitter[2] == "(R)":
        splitter[2] = "Rare"
    if splitter[2] == "(I)":
        splitter[2] = "Introduced"
    if splitter[2] == "(E)":
        splitter[2] = "Extinct"
    if splitter[2] == "(P)":
        splitter[2] = "Provisional"
    if splitter[2] == "(native to the southwestern U.S.; introduced in the east)":
        splitter[2] = "Introduced"
    genus.append(splitter[1])
    info.append(splitter[2])
df = pd.DataFrame()
df["names"] = names; df["genus"] = genus; df["info"] = info
df

,names,genus,info
0,Black-bellied whistling-duck,Dendrocygna autumnalis,None
1,Fulvous whistling-duck,Dendrocygna bicolor,Rare
2,Snow goose,Anser caerulescens,None
3,Ross's goose,Anser rossii,None
4,Greater white-fronted goose,Anser albifrons,None
...,...,...,...
428,Blue grosbeak,Passerina caerulea,None
429,Indigo bunting,Passerina cyanea,None
430,Lazuli bunting,Passerina amoena,Rare
431,Painted bunting,Passerina ciris,None


# Web Scraping information

The below is web scraping code used to pull image for each bird from the internet. First we import the libraries then we just go forth


In [96]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
import os
import time
from selenium.webdriver.common.by import By

In [97]:
def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

        # build the google query

    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)
        scroll_to_end(wd)
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements(By.CLASS_NAME,"Q4LuWd")
        number_results = len(thumbnail_results)

#         print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = wd.find_elements(By.CLASS_NAME,'Q4LuWd')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
       

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


In [98]:
def persist_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url).content
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")
        
    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [102]:
def search_and_download(search_term: str, driver_path: str, target_path='./images', number_images=10):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    options = Options()
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-gpu")
    options.add_argument("--headless")
    
    
    with webdriver.Chrome(options = options) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    counter = 0
    for elem in res:
        persist_image(target_folder, elem, counter)
        counter += 1

In [103]:
for i in df["names"]:
    print("searching for images of {}".format(i))
    search_and_download(search_term=i, driver_path='./chromedriver', number_images = 100)

searching for images of Black-bellied whistling-duck
Found: 100 search results. Extracting links from 0:100
Found: 104 image links, done!
searching for images of Fulvous whistling-duck
Found: 100 search results. Extracting links from 0:100
Found: 106 image links, done!
searching for images of Snow goose
Found: 100 search results. Extracting links from 0:100
Found: 103 image links, done!


KeyboardInterrupt: 